## **MIMIC-III**: Pre-processing

This notebook contains the pre-processing of the [MIMIC-III](https://physionet.org/content/mimiciii/1.4/) dataset. 

In [31]:
from data import *
from utils import *
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

from data import *

MIMIC_PATH = '../data/mimic-iv/'

## 1. **Tabular data**

[MIMIC-IV Documentation](https://mimic.mit.edu/docs/iv/). 

**Tables that we use**

* `Admissions`
* `Patients`
* `Services`
* `emar`
* `d_hcpcs.csv.gz`
* `hpcsevents`
* `labevents`
* `microbiologyevents`



### Preprocess admissions and patients

In [61]:
# Load admissions, patients, and icustays
admissions = pd.read_csv(MIMIC_PATH+'admissions.csv.gz')
patients = pd.read_csv(MIMIC_PATH+'patients.csv.gz')

# We remove the columns DISCHTIME (discharge time), DEATHTIME (death time), DISCHARGE_LOCATION (discharge location), EDREGTIME (registered on ER), EDOUTTIME(discharged from ER), DIAGNOSIS, HOSPITAL_EXPIRE_FLAG (died in hospital)

admissions_cleaned = admissions.drop(columns=['dischtime', 'deathtime', 'discharge_location', 'edregtime', 'edouttime', 'hospital_expire_flag'])
# do it without capital letters
patients_cleaned = patients.drop(columns=['dod'])

# Merge the two tables
tabular_data = pd.merge(admissions_cleaned, patients_cleaned, on='subject_id', how='left')

# The admittime has the format YYYY-MM-DD HH:MM:SS, convert it to DateTime
tabular_data['admittime'] = pd.to_datetime(tabular_data['admittime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check the types of the columns
tabular_data


,subject_id,hadm_id,admittime,admission_type,admit_provider_id,admission_location,insurance,language,marital_status,race,gender,anchor_age,anchor_year,anchor_year_group
0,10000032,22595853,2180-05-06 22:23:00,URGENT,P874LG,TRANSFER FROM HOSPITAL,Other,ENGLISH,WIDOWED,WHITE,F,52,2180,2014 - 2016
1,10000032,22841357,2180-06-26 18:27:00,EW EMER.,P09Q6Y,EMERGENCY ROOM,Medicaid,ENGLISH,WIDOWED,WHITE,F,52,2180,2014 - 2016
2,10000032,25742920,2180-08-05 23:44:00,EW EMER.,P60CC5,EMERGENCY ROOM,Medicaid,ENGLISH,WIDOWED,WHITE,F,52,2180,2014 - 2016
3,10000032,29079034,2180-07-23 12:35:00,EW EMER.,P30KEH,EMERGENCY ROOM,Medicaid,ENGLISH,WIDOWED,WHITE,F,52,2180,2014 - 2016
4,10000068,25022803,2160-03-03 23:16:00,EU OBSERVATION,P51VDL,EMERGENCY ROOM,Other,ENGLISH,SINGLE,WHITE,F,19,2160,2008 - 2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,Other,ENGLISH,SINGLE,WHITE,F,46,2147,2017 - 2019
431227,19999828,29734428,2147-07-18 16:23:00,EW EMER.,P16C7J,PHYSICIAN REFERRAL,Other,ENGLISH,SINGLE,WHITE,F,46,2147,2017 - 2019
431228,19999840,21033226,2164-09-10 13:47:00,EW EMER.,P58A9J,EMERGENCY ROOM,Other,ENGLISH,WIDOWED,WHITE,M,58,2164,2008 - 2010
431229,19999840,26071774,2164-07-25 00:27:00,EW EMER.,P506DE,EMERGENCY ROOM,Other,ENGLISH,WIDOWED,WHITE,M,58,2164,2008 - 2010


### Preprocess metadata images

In [74]:
PATH_METADATA = '../data/mimic-cxr-jpg/mimic-cxr-2.0.0-metadata.csv.gz'
metadata = pd.read_csv(PATH_METADATA)
len_old = len(metadata)
# Keep only the study ideas with exactly 2 images
groups = metadata.groupby('study_id')
study_ids = []
for study_id, group in groups:
    if len(group) > 1:
        # Keep study_id if there is a ViewPosition PA and LATERAL
        if 'PA' in group['ViewPosition'].values and 'LATERAL' in group['ViewPosition'].values:
            study_ids.append(study_id)
metadata = metadata[metadata['study_id'].isin(study_ids)]
# Keep only the images with ViewPosition PA and LATERAL
metadata = metadata[metadata['ViewPosition'].isin(['PA', 'LATERAL'])]
# Group by study_id and by ViewPosition and keep only the first image
metadata = metadata.groupby(['study_id', 'ViewPosition']).first().reset_index()

len_new = len(metadata)
print(f'Number of studies with Lateral and PA image: {len_new} ({len_new/len_old*100:.2f}%)')

Number of studies with Lateral and PA image: 114172 (30.28%)


In [75]:
metadata_time = pd.DataFrame(columns=['subject_id', 'StudyDate', 'StudyTime', 'dicom_id', 'study_id', 'ViewPosition'])
# We only keep the columns subject_id, StudyDate, StudyTime, dicom_id, study_id
metadata_time['dicom_id'] = metadata['dicom_id']
metadata_time['study_id'] = metadata['study_id']
metadata_time['ViewPosition'] = metadata['ViewPosition']
metadata_time['subject_id'] = metadata['subject_id']
# The study data is in the form YYYYMMDD, we need to convert it to DateTime
metadata_time['StudyDate'] = pd.to_datetime(metadata['StudyDate'], format='%Y%m%d').dt.date
# Studytime is in the form HHMMSS.SSS or HHMMSS or HHMMSS.SS or HHMMSS.S, remove the fraction of seconds, round to the nearest second, it is a numpy float
metadata_time['StudyTime'] = metadata['StudyTime'].astype(str).str.split('.').str[0]
metadata_time['StudyTime'] = pd.to_datetime(metadata['StudyTime'], format='%H%M%S', errors='coerce').dt.time

old_len = len(metadata_time)
#Drop rows where either StudyDate or StudyTime is NaT
metadata_time = metadata_time.dropna(subset=['StudyDate', 'StudyTime'])
new_len = len(metadata_time)
print(f'Dropped {old_len - new_len} rows')
# Now combine the date and time into one column as DateTime
metadata_time['StudyDateTime'] = pd.to_datetime(metadata_time['StudyDate'].astype(str) + ' ' + metadata_time['StudyTime'].astype(str))
# Drop the columns StudyDate and StudyTime
metadata_time = metadata_time.drop(columns=['StudyDate', 'StudyTime'])
metadata_time

Dropped 68 rows


,subject_id,dicom_id,study_id,ViewPosition,StudyDateTime
0,19995127,1ff9650f-4cb5af18-f6caef33-e53686b7-983cca76,50000028,LATERAL,2138-03-07 18:51:31
1,19995127,33708b7f-979612cb-46e1424c-318354bc-6f043189,50000028,PA,2138-03-07 18:51:31
2,14444780,c7dadf13-58bc3fd8-7e7f4ba4-9ac13218-43aae1b2,50000186,LATERAL,2117-11-30 14:45:37
3,14444780,93bcf53f-7c91b330-3738f326-4d31769d-6cff6fe5,50000186,PA,2117-11-30 14:45:37
4,11550925,f605b192-2e612578-c5c95dc3-b9d6d13b-e0eee500,50000230,LATERAL,2172-09-04 16:21:59
...,...,...,...,...,...
114167,16233094,ae9b3328-1639be10-77b351b4-ab8ac17a-183095de,59999431,PA,2145-10-31 19:38:55
114168,13966009,7c1d5404-71a99f61-660f988c-59c4416a-a9bcbc83,59999472,LATERAL,2197-04-15 23:33:02
114169,13966009,1408eab2-99933dbb-b39e5336-3f82460f-5daaa269,59999472,PA,2197-04-15 23:33:02
114170,16352630,578a176c-ce729c63-ed8441fc-67506d50-e44d920f,59999636,LATERAL,2143-11-14 20:01:59


In [82]:
# Now we want to merge with tabular data based on subject_id, if there are multiple rows in tabular data with the same subject_id, we keep the with the latest admittime, where the admittime is before the StudyDateTime
# We first merge the two tables
merged = pd.merge(metadata_time, tabular_data, on='subject_id', how='left')
# We keep only the rows where the StudyDateTime is after the admittime
merged = merged[merged['StudyDateTime'] >= merged['admittime']]
# We group by dicom_id and keep the row with the latest admittime
merged = merged.sort_values('admittime').groupby('dicom_id').last().reset_index()
print('Number of pictures: ', len(merged))
print('Number of unique subject_id: ', len(merged['subject_id'].unique()))
print('Number of unique study_id: ', len(merged['study_id'].unique()))
merged.head()



Number of pictures:  66864
Number of unique subject_id:  18995
Number of unique study_id:  33432


,dicom_id,subject_id,study_id,ViewPosition,StudyDateTime,hadm_id,admittime,admission_type,admit_provider_id,admission_location,insurance,language,marital_status,race,gender,anchor_age,anchor_year,anchor_year_group
0,00000218-9fb20d4e-86045713-8013e08b-0d5bebba,17191670,51332563,LATERAL,2176-10-24 21:11:22,22802102.0,2176-10-24 09:29:00,EU OBSERVATION,P29CGZ,EMERGENCY ROOM,Medicaid,ENGLISH,SINGLE,WHITE,M,45.0,2171.0,2008 - 2010
1,0000c2f5-f02f9f3c-1ed14642-958de0ad-d6ce4d20,17744443,56580856,PA,2128-03-19 08:05:20,22867350.0,2128-03-19 07:52:00,EU OBSERVATION,P23URY,EMERGENCY ROOM,Other,ENGLISH,SINGLE,WHITE,M,29.0,2128.0,2011 - 2013
2,0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab,19298916,50189753,PA,2193-02-22 05:28:41,25272188.0,2192-11-29 10:35:00,EU OBSERVATION,P59HPG,EMERGENCY ROOM,Other,ENGLISH,DIVORCED,HISPANIC/LATINO - PUERTO RICAN,F,66.0,2188.0,2011 - 2013
3,0002ba95-e4325b54-a0e16b3b-3cf6c9d4-1b1c1910,18958209,56891628,LATERAL,2129-12-16 16:31:17,26018739.0,2129-09-04 19:26:00,EU OBSERVATION,P42K8Q,EMERGENCY ROOM,Other,ENGLISH,MARRIED,WHITE,M,47.0,2123.0,2008 - 2010
4,00039aac-942aea85-a4f7cd1c-faed6733-c8701b02,10900906,53677475,LATERAL,2160-09-13 18:39:54,23329650.0,2160-09-02 22:34:00,EW EMER.,P77SO2,EMERGENCY ROOM,Other,ENGLISH,MARRIED,WHITE,M,46.0,2156.0,2008 - 2010


In [80]:
services = pd.read_csv(MIMIC_PATH+'services.csv.gz')
services

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10000032,22595853,2180-05-06 22:24:57,NaN,MED
1,10000032,22841357,2180-06-26 18:28:08,NaN,MED
2,10000032,25742920,2180-08-05 23:44:50,NaN,MED
3,10000032,29079034,2180-07-23 12:36:04,NaN,MED
4,10000068,25022803,2160-03-03 23:17:17,NaN,MED
...,...,...,...,...,...
468024,19999828,29734428,2147-07-18 16:24:15,NaN,SURG
468025,19999840,21033226,2164-09-10 13:49:01,NaN,NMED
468026,19999840,21033226,2164-09-14 10:40:18,NMED,MED
468027,19999840,26071774,2164-07-25 00:27:59,NaN,NMED


### Preprocess services

In [86]:
services = pd.read_csv(MIMIC_PATH+'services.csv.gz')
unique_services = services['curr_service'].unique()
services['transfertime'] = pd.to_datetime(services['transfertime'], format='%Y-%m-%d %H:%M:%S').dt.date
# Merge the services with the tabular data, on hadm_id and subject_id
tabular_metadata_services = pd.merge(merged, services, on=['hadm_id', 'subject_id'], how='left')

# Now only keep the rows where the transfertime is before the StudyDateTime
tabular_metadata_services = tabular_metadata_services[tabular_metadata_services['transfertime'] <= tabular_metadata_services['StudyDateTime']]

# Group by dicom_id and take the row with the latest transfertime
tabular_metadata_services = tabular_metadata_services.sort_values('transfertime').groupby('dicom_id').last().reset_index()

for service in unique_services:
    #initiate the column with 0
    tabular_metadata_services[service] = 0

# Set the service to 1 if it is in the curr_service column
for index, row in tabular_metadata_services.iterrows():
    tabular_metadata_services.loc[index, row['curr_service']] = 1

# Drop the curr_service column, the transfertime column and the prev_service column

tabular_metadata_services = tabular_metadata_services.drop(columns=['curr_service', 'transfertime', 'prev_service'])
tabular_metadata_services

,dicom_id,subject_id,study_id,ViewPosition,StudyDateTime,hadm_id,admittime,admission_type,admit_provider_id,admission_location,...,NSURG,PSURG,NMED,GYN,VSURG,CSURG,TSURG,TRAUM,EYE,DENT
0,0000c2f5-f02f9f3c-1ed14642-958de0ad-d6ce4d20,17744443,56580856,PA,2128-03-19 08:05:20,22867350.0,2128-03-19 07:52:00,EU OBSERVATION,P23URY,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
1,0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab,19298916,50189753,PA,2193-02-22 05:28:41,25272188.0,2192-11-29 10:35:00,EU OBSERVATION,P59HPG,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
2,0002ba95-e4325b54-a0e16b3b-3cf6c9d4-1b1c1910,18958209,56891628,LATERAL,2129-12-16 16:31:17,26018739.0,2129-09-04 19:26:00,EU OBSERVATION,P42K8Q,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
3,00039aac-942aea85-a4f7cd1c-faed6733-c8701b02,10900906,53677475,LATERAL,2160-09-13 18:39:54,23329650.0,2160-09-02 22:34:00,EW EMER.,P77SO2,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
4,0003db13-ec18c5d8-d922bc62-b39ab6fb-60b56b9c,14549454,58100756,LATERAL,2145-02-26 18:35:28,24351783.0,2141-09-27 23:15:00,EW EMER.,P30SZF,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66687,fff26570-3cc431ca-d9ac0643-727b046d-00a41963,15243252,50214443,PA,2134-11-15 23:45:49,24139795.0,2133-11-12 02:26:00,EW EMER.,P816G0,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
66688,fff33ebb-05690951-7f27dcfc-648db54e-7036d94c,13271160,54593756,PA,2128-02-06 09:10:10,20872525.0,2128-01-21 17:27:00,EW EMER.,P15D9X,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
66689,fff6a589-7b47bcac-56195851-53d74c55-97c5d30a,15403581,53793119,PA,2179-10-07 14:29:57,26811647.0,2178-08-20 08:19:00,EW EMER.,P23URY,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0
66690,fffb4a2d-e8766181-fb54156a-9a2fd941-e87009ef,14361828,51569492,PA,2197-06-17 17:27:39,29563896.0,2197-06-07 18:14:00,EW EMER.,P89OYR,EMERGENCY ROOM,...,0,0,0,0,0,0,0,0,0,0


In [87]:
# Save to csv
tabular_metadata_services.to_csv('../data/mimic-iv/tabular_metadata_services.csv.gz', index=False)

##

### Exploratory analysis

In [29]:
PATH_LABELS = '../data/mimic-cxr-jpg/mimic-cxr-2.0.0-chexpert.csv.gz'

# We will one-hot encode the labels
labels = pd.read_csv(PATH_LABELS)
labels


,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227822,19999442,58708861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
227823,19999733,57132437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
227824,19999987,55368167,1.0,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
227825,19999987,58621812,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## 2. **Images data**


The mimic-cxr-2.0.0-metadata.csv.gz file contains useful meta-data derived from the original DICOM files in MIMIC-CXR. The columns are:

* **dicom_id** - An identifier for the DICOM file. The stem of each JPG image filename is equal to the dicom_id.
* **PerformedProcedureStepDescription** - The type of study performed ("CHEST (PA AND LAT)", "CHEST (PORTABLE AP)", etc).
* **ViewPosition** - The orientation in which the chest radiograph was taken ("AP", "PA", "LATERAL", etc).
* **Rows** - The height of the image in pixels.
* **Columns** - The width of the image in pixels.
* **StudyDate** - An anonymized date for the radiographic study. All images from the same study will have the same date and time. Dates are anonymized, but chronologically consistent for each patient. Intervals between two scans have not been modified during de-identification.
* **StudyTime** - The time of the study in hours, minutes, seconds, and fractional seconds. The time of the study was not modified during de-identification.
* **ProcedureCodeSequence_CodeMeaning** - The human readable description of the coded procedure (e.g. "CHEST (PA AND LAT)". Descriptions follow Simon-Leeming codes [11].
* **ViewCodeSequence_CodeMeaning** - The human readable description of the coded view orientation for the image (e.g. "postero-anterior", "antero-posterior", "lateral").
* **PatientOrientationCodeSequence_CodeMeaning** - The human readable description of the patient orientation during the image acquisition. Three values are possible: "Erect", "Recumbent", or a null value (missing).

In [ ]:
# Info from the images
info_jpg = pd.read_csv('../fake_data/mimic-cxr-2.0.0-metadata.csv')
info_jpg.head()

Note that "No Finding" is the absence of any of the 13 descriptive labels and a check that the text does not mention a specified set of other common findings beyond those covered by the descriptive labels. Each label column contains one of four values: 1.0, -1.0, 0.0, or missing. These labels have the following interpretation:

* **1.0** - The label was positively mentioned in the associated study, and is present in one or more of the corresponding images e.g. "A large pleural effusion"
* **0.0** - The label was negatively mentioned in the associated study, and therefore should not be present in any of the corresponding images e.g. "No pneumothorax."
* **-1.0** - The label was either: (1) Explicit uncertainty or (2) Ambiguous language
* **Missing** (empty element) - No mention of the label was made in the report

In [ ]:
labels_data = pd.read_csv('../fake_data/mimic-cxr-2.0.0-chexpert.csv')
labels_data.head()

In [ ]:
def list_images(base_path):
    """
    Recursively lists all image files starting from the base path.
    Assumes that images have extensions typical for image files (e.g., .jpg, .jpeg, .png).
    """
    image_files = []
    for subdir, dirs, files in os.walk(base_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(subdir, file))
    return image_files

image_files = list_images('../fake_data/files')
image_files

In [ ]:
image_labels_mapping = create_image_labels_mapping(image_files, labels_data, info_jpg)
        
image_labels_mapping[image_files[0]]    

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image, ImageOps

class MedicalImagesDataset(Dataset):
    def __init__(self, data_dict, size=224, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.size = size
        self.classes = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 
                        'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 
                        'Lung Opacity', 'No Finding', 'Pleural Effusion', 
                        'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']
        # Organize paths by subject_id and study_id
        self.organized_paths = self._organize_paths()
        # Filter out pairs where both images are None
        self.organized_paths = {k: v for k, v in self.organized_paths.items() if v['PA'] is not None or v['Lateral'] is not None}

    def _organize_paths(self):
        organized = {}
        for path in self.data_dict.keys():
            parts = path.split(os.sep)
            subject_id = parts[-3][1:]
            study_id = parts[-2][1:]
            key = (subject_id, study_id)

            if key not in organized:
                organized[key] = {'PA': None, 'Lateral': None}

            view_position = self.data_dict[path]['ViewPosition']
            if view_position in ['PA', 'Lateral']:
                organized[key][view_position] = path

        return organized

    def __len__(self):
        return len(self.organized_paths)

    def _load_and_process_image(self, path):
        if path:
            image = Image.open(path).convert('RGB')
        else:
            # Create a blank (black) image if path is None
            image = Image.new('RGB', (self.size, self.size))

        image = image.resize((self.size, self.size))

        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image)

        return image

    def __getitem__(self, idx):
        subject_study_pair = list(self.organized_paths.keys())[idx]
        pa_path = self.organized_paths[subject_study_pair]['PA']
        lateral_path = self.organized_paths[subject_study_pair]['Lateral']

        # Load and process PA and Lateral images
        pa_image = self._load_and_process_image(pa_path)
        lateral_image = self._load_and_process_image(lateral_path)

        # Use one of the available paths to get labels (assuming they are the same for both views)
        labels_path = pa_path if pa_path else lateral_path

        if not labels_path:
            # Skip this patient if both PA and Lateral images are missing
            return None

        labels = self.data_dict[labels_path]
        label_values = [labels[class_name] if not np.isnan(labels[class_name]) else 0 for class_name in self.classes]
        label_tensor = torch.tensor(label_values, dtype=torch.float32)

        return pa_image, lateral_image, label_tensor

# Transform definition (if needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MedicalImagesDataset(image_labels_mapping, transform=transform)
dataloaders = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

for pa_images, lateral_images, labels in dataloaders:
    print(pa_images.shape)
    print(lateral_images.shape)
    print(labels.shape)
    break


In [ ]:
# Iterate over batches of image,label pairs
for i, (images, labels, view) in enumerate(dataloader):
    print(images.shape)
    print(labels.shape)
    print(view)
    break